In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install ktrain
import ktrain
from ktrain import text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 36.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 64.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 45.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.41.3-py3-none-any.whl size=25316960 sha256=84e471c8e88382c8b02945e1fe5baf32fa1766de6c369e3986a6945100460720
  Stored in directory: /root/.cache/pip/wheels/fc/76/11/5b953090eebf531f660948a30cd26e70260619f6480f186a5a
  Created wheel for keras_bert: filename=

# Bert

In [3]:
import pandas as pd
import numpy as np
import re
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv("/content/drive/MyDrive/Major Project Online Hate Speech - English/dataset/mam_data.csv")

In [5]:
# Assuming X and y are your data and target variable, respectively
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)


In [6]:
X_train = data_train.comment.tolist()
X_test = data_test.comment.tolist()

y_train = data_train.isHate.tolist()
y_test = data_test.isHate.tolist()

data = pd.concat([data_train, data_test], ignore_index=True)

class_names = ['hate', 'non hate']

In [7]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350,
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [8]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [9]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [11]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
5558/5558 [==============================] - 1087s 196ms/step - loss: 0.1366 - accuracy: 0.9522 - val_loss: 0.1021 - val_accuracy: 0.9617


In [12]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

261/261 [==============================] - 73s 263ms/step
              precision    recall  f1-score   support

        hate       0.96      0.97      0.97      4677
    non hate       0.96      0.95      0.96      3660

    accuracy                           0.96      8337
   macro avg       0.96      0.96      0.96      8337
weighted avg       0.96      0.96      0.96      8337



array([[4537,  140],
       [ 179, 3481]])

In [13]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['hate', 'non hate']

In [14]:
import time

message = 'Well im glad that i live in Serbia, migrants and islamists are not welcome here!'

start_time = time.time()
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: hate (0.11)
